# Matrix editing
- remove no_host records
- select first n most abundant hosts
- feature selection with variance threshold on 10% of the least abundant species from selection

In [1]:
# imports
import pandas as pd
from sklearn.feature_selection import VarianceThreshold

In [2]:
# parameters
matrix_file = '/data/projects/kimona/data_18-03-08/011_matrix.tsv'
hosts_file = '/data/projects/kimona/data_18-03-08/004_hosts'
hosts_counts_file = '/data/projects/kimona/data_18-03-08/004_hosts.counts'
out = '/data/projects/kimona/data_18-03-08/011_edited_matrix.tsv'

# number of most abundant hosts to count in
n = 8

# threshold level as in description
level = 0.1

In [3]:
# matrix loading
matrix = pd.read_csv(matrix_file, sep='\t', header=0, index_col=0)

In [12]:
matrix

,Cluster_0,Cluster_1,Cluster_2,Cluster_3,Cluster_4,Cluster_5,Cluster_6,Cluster_7,Cluster_8,Cluster_9,...,Cluster_15007,Cluster_15008,Cluster_15009,Cluster_15010,Cluster_15011,Cluster_15012,Cluster_15013,Cluster_15014,Cluster_15015,Cluster_15016
phage0000009,0,0,0,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
phage0000012,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
phage0000022,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
phage0000023,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
phage0000024,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
phage0000025,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
phage0000028,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
phage0000029,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
phage0000030,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
phage0000031,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# removing no_host records
with open(hosts_file) as f:
    hosts = f.readlines()

phages_to_drop = []

for record in hosts:
    
    phage = record.split('\t')[0]
    hosts_string = record.split('\t')[1].strip()
    
    if hosts_string == 'no_host':
        phages_to_drop.append(phage)
        
dropped = matrix.drop(phages_to_drop)

print(dropped.shape)

ValueError: labels ['phage0000007' 'phage0000147' 'phage0000148' 'phage0000149' 'phage0000150'
 'phage0000151' 'phage0000152' 'phage0000219' 'phage0000220' 'phage0000221'
 'phage0000259' 'phage0000297' 'phage0000298' 'phage0000299' 'phage0000311'
 'phage0000312' 'phage0000313' 'phage0000314' 'phage0000318' 'phage0000326'
 'phage0000327' 'phage0000403' 'phage0000725' 'phage0000903' 'phage0000904'
 'phage0000905' 'phage0000906' 'phage0000907' 'phage0000910' 'phage0001288'
 'phage0001348' 'phage0001349' 'phage0001385' 'phage0001468' 'phage0001492'
 'phage0001493' 'phage0001522' 'phage0001523' 'phage0001524' 'phage0001740'
 'phage0001756' 'phage0001766' 'phage0001767' 'phage0001768' 'phage0001769'
 'phage0001770' 'phage0001800' 'phage0001801' 'phage0001802' 'phage0001803'
 'phage0001804' 'phage0001805' 'phage0001806' 'phage0001807' 'phage0001808'
 'phage0001809' 'phage0001810' 'phage0001811' 'phage0001812' 'phage0001813'
 'phage0001814' 'phage0001815' 'phage0001816' 'phage0001817' 'phage00

In [7]:
# selecting first n most abundant hosts
with open(hosts_counts_file) as f:
    hosts_counts = f.readlines()

strings = []
counts = []

for line in hosts_counts:
    string = line.split()[1].strip()
    count = int(line.split()[0].strip())
    
    if string == 'no_host':
        n += 1
    else:
        counts.append(count)
    
    strings.append(string)
    if len(strings) == n:
        break
        
print(strings)
print(counts)

['mycobac', 'no_host', 'strepto', 'escheri', 'gordoni', 'arthrob', 'pseudom', 'lactoco', 'staphyl', 'bacillu']
[1619, 354, 323, 293, 240, 236, 219, 184, 169]


In [18]:
# removing other than strings records
phages_to_drop = []

for record in hosts:
    
    phage = record.split('\t')[0]
    hosts_string = record.split('\t')[1].strip()
    to_delete = True
    
    for string in strings:
        if string in hosts_string:
            to_delete = False
            
    if to_delete:
        phages_to_drop.append(phage)
        
pure_matrix = dropped.drop(phages_to_drop)

In [19]:
# feature selection
min_count = min(counts)

var_threshold = (min_count*level)/pure_matrix.shape[0]

sel = VarianceThreshold(threshold=var_threshold * (1 - var_threshold))
trained_sel = sel.fit(pure_matrix)
final_matrix = pure_matrix.iloc[:,trained_sel.get_support(indices=True)]

In [21]:
# saving new dataframe
final_matrix.to_csv(out, sep='\t')